In [ ]:
import requests
from bs4 import BeautifulSoup

def get_html_content(url):
    response = requests.get(url)
    if response.status_code == 200:
        return response.text
    else:
        return None

def extract_article_title(html_content):
    soup = BeautifulSoup(html_content, 'html.parser')
    title = soup.find('h1', {'class': 'firstHeading'}).text
    return title

def extract_article_text_with_headings(html_content):
    soup = BeautifulSoup(html_content, 'html.parser')
    content = {}
    
    current_heading = None
    paragraphs = soup.find_all(['p', 'h2', 'h3', 'h4', 'h5', 'h6'])
    
    for element in paragraphs:
        if element.name.startswith('h'):
            current_heading = element.text
            content[current_heading] = []
        elif current_heading:
            content[current_heading].append(element.text)
            
    return content

def collect_redirect_links(html_content):
    soup = BeautifulSoup(html_content, 'html.parser')
    redirect_links = []

    for link in soup.find_all('a', href=True):
        if link['href'].startswith('/wiki/'):
            redirect_links.append(link['href'])

    return redirect_links

def process_wikipedia_page(url):
    html_content = get_html_content(url)
    if not html_content:
        return None
    
    title = extract_article_title(html_content)
    article_text_with_headings = extract_article_text_with_headings(html_content)
    redirect_links = collect_redirect_links(html_content)
    
    result = {
        'title': title,
        'article_text_with_headings': article_text_with_headings,
        'redirect_links': redirect_links
    }
    
    return result

# Test the function on a Wikipedia page of your choice
wikipedia_link = 'https://en.wikipedia.org/wiki/Python_(programming_language)'
result = process_wikipedia_page(wikipedia_link)
print(result)
